

# Importing libraries



In [1]:

# ***************************************************************************
import numpy as np
import re

import nltk
from nltk.tokenize import sent_tokenize
nltk.download('punkt')

import spacy

from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min

import bs4 as BeautifulSoup
import urllib.request  
# ***************************************************************************


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
!python -m spacy download es_core_news_md

     |████████████████████████████████| 73.4MB 1.4MB/s 
  Created wheel for es-core-news-md: filename=es_core_news_md-2.1.0-cp36-none-any.whl size=74602272 sha256=17ef4fc4e033a3d66955f082ad55affc0328d6838fa1be80cce2443d1a9bbf16
  Stored in directory: /tmp/pip-ephem-wheel-cache-7kn7v6la/wheels/20/f7/6b/7a1ba56f009b05386d123ea088b56635594a046acd5cfdd2a2
Successfully built es-core-news-md
✔ Download and installation successful
You can now load the model via spacy.load('es_core_news_md')


#Fetching content and preprocessing

In [0]:


#fetching the content from the URL
fetched_data = urllib.request.urlopen('https://es.wikipedia.org/wiki/Prote%C3%ADna')

article_read = fetched_data.read()

#parsing the URL content and storing in a variable
article_parsed = BeautifulSoup.BeautifulSoup(article_read,'html.parser')

#returning <p> tags
paragraphs = article_parsed.find_all('p')

article_content = ''

#looping through the paragraphs and adding them to the variable
for p in paragraphs:  
    article_content += p.text

In [45]:
article_content

'Las proteínas (< en francés: protéine < en griego: πρωτεῖος\xa0[proteios],\xa0‘prominente, de primera calidad’?)[1]\u200b o prótidos[2]\u200b son macromoléculas formadas por cadenas lineales de aminoácidos. \nEstas ￼se ensamblan de diversas formas lo que les permite participar como los principales componentes estructurales de las células y los tejidos. Por este motivo el crecimiento, la reparación y el mantenimiento del organismo dependen de ellas. [3]\u200b \nPor sus propiedades fisicoquímicas, las proteínas se pueden clasificar en proteínas simples (holoproteidos), formadas solo por aminoácidos o sus derivados; proteínas conjugadas (heteroproteidos), formadas por aminoácidos acompañados de sustancias diversas, y proteínas derivadas, sustancias formadas por desnaturalización y desdoblamiento de las anteriores. Están constituidas por unidades estructurales llamados polímeros[4]\u200b. Las proteínas son necesarias para la vida, sobre todo por su función plástica (constituyen el 80\xa0%

In [46]:
retext = re.split('\n',article_content)
len(retext)

114

In [47]:
for par in retext:
  print(par)

Las proteínas (< en francés: protéine < en griego: πρωτεῖος [proteios], ‘prominente, de primera calidad’?)[1]​ o prótidos[2]​ son macromoléculas formadas por cadenas lineales de aminoácidos. 
Estas ￼se ensamblan de diversas formas lo que les permite participar como los principales componentes estructurales de las células y los tejidos. Por este motivo el crecimiento, la reparación y el mantenimiento del organismo dependen de ellas. [3]​ 
Por sus propiedades fisicoquímicas, las proteínas se pueden clasificar en proteínas simples (holoproteidos), formadas solo por aminoácidos o sus derivados; proteínas conjugadas (heteroproteidos), formadas por aminoácidos acompañados de sustancias diversas, y proteínas derivadas, sustancias formadas por desnaturalización y desdoblamiento de las anteriores. Están constituidas por unidades estructurales llamados polímeros[4]​. Las proteínas son necesarias para la vida, sobre todo por su función plástica (constituyen el 80 % del protoplasma deshidratado de

In [0]:
def split_sentences(par):
    """
    Splits the paragraph into individual sentences
    """
    sentences = sent_tokenize(par)
    for j in reversed(range(len(sentences))):
      sent = sentences[j]
      sentences[j] = sent.strip()
      if sent == '':
        sentences.pop(j)
    return sentences

In [50]:
par1 = retext[12]
par1

'Las proteínas se ensamblan a partir de sus aminoácidos utilizando la información codificada en los genes. Cada proteína tiene su propia secuencia de aminoácidos que está especificada por la secuencia de nucleótidos del gen que la codifica. El código genético está formado por un conjunto de tri-nucleótidos denominados codones. Cada codón (combinación de tres nucleótidos) designa un aminoácido, por ejemplo AUG (adenina-uracilo-guanina) es el código para la metionina. Como el ADN contiene cuatro nucleótidos distintos, el número total de codones posibles es 64; por lo tanto, existe cierta redundancia en el código genético, estando algunos aminoácidos codificados por más de un codón. Los genes codificados en el ADN se transcriben primero en ARN pre-mensajero mediante proteínas como la ARN polimerasa. La mayor parte de los organismos procesan entonces este pre-ARNm (también conocido como tránscrito primario) utilizando varias formas de modificación post-transcripcional para formar ARNm madu

In [51]:
par2 = split_sentences(par1)
par2

['Las proteínas se ensamblan a partir de sus aminoácidos utilizando la información codificada en los genes.',
 'Cada proteína tiene su propia secuencia de aminoácidos que está especificada por la secuencia de nucleótidos del gen que la codifica.',
 'El código genético está formado por un conjunto de tri-nucleótidos denominados codones.',
 'Cada codón (combinación de tres nucleótidos) designa un aminoácido, por ejemplo AUG (adenina-uracilo-guanina) es el código para la metionina.',
 'Como el ADN contiene cuatro nucleótidos distintos, el número total de codones posibles es 64; por lo tanto, existe cierta redundancia en el código genético, estando algunos aminoácidos codificados por más de un codón.',
 'Los genes codificados en el ADN se transcriben primero en ARN pre-mensajero mediante proteínas como la ARN polimerasa.',
 'La mayor parte de los organismos procesan entonces este pre-ARNm (también conocido como tránscrito primario) utilizando varias formas de modificación post-transcripcio

#Encoding sentences and using K-means

In [0]:
nlp = spacy.load("es_core_news_md")  # Spanish encoding model


In [0]:
def sent_encode(par):
    """
    Obtains sentence embeddings for each sentence in the paragraph
    """
    enc_par = []
    
    for i in range(len(par)):

      print('Encoding sentence ' + str(i))
      enc_sent = nlp(par[i]).vector
      enc_par.append(enc_sent)

    return enc_par

In [52]:
par3 = sent_encode(par2)
par3

Encoding sentence 0
Encoding sentence 1
Encoding sentence 2
Encoding sentence 3
Encoding sentence 4
Encoding sentence 5
Encoding sentence 6
Encoding sentence 7
Encoding sentence 8
Encoding sentence 9
Encoding sentence 10


[array([-0.0150147 , -1.0762547 ,  0.03071171, -0.48124352,  0.3425341 ,
        -0.07702905, -0.15837476, -0.34623918, -0.02209864,  3.1386821 ,
         0.5085597 ,  0.37686336,  0.05889688, -0.3501957 ,  1.3512353 ,
        -0.06457248,  0.035643  ,  0.109364  , -0.27256873, -0.13137819,
         1.0245657 ,  0.19740635, -0.030686  , -0.3189381 , -0.27628326,
         0.24071014, -0.8532512 ,  1.7748352 ,  0.7212638 , -0.01116618,
         1.7862251 , -0.44874296, -1.704863  , -0.34440655,  0.02198147,
         0.16672239,  0.6018072 , -0.5963396 ,  0.575108  , -0.00607878,
         0.08633947, -0.36125684, -0.00393353,  0.03392623,  0.9531573 ,
         0.20252813, -0.15829328, -2.1899488 , -0.1675414 , -0.5235251 ],
       dtype=float32),
 array([-0.45335427, -1.3066441 ,  0.03853182, -0.5745421 ,  0.37614238,
        -0.02510436, -0.20563918, -0.253313  , -0.22790147,  3.2233796 ,
         0.4581275 ,  0.6017053 ,  0.1159041 ,  0.05008692,  1.2391926 ,
        -0.11799955,  0.359

In [73]:

n_clusters = int(np.ceil(len(par3)*0.5))
kmeans = KMeans(n_clusters=n_clusters, random_state=0)
kmeans = kmeans.fit(par3)
avg = []
closest = []
for j in range(n_clusters):
  idx = np.where(kmeans.labels_ == j)[0]
  avg.append(np.mean(idx))
closest, _ = pairwise_distances_argmin_min(kmeans.cluster_centers_, par3)
closest2 = np.sort(closest)
#ordering = sorted(range(n_clusters), key=lambda k: avg[k])
summary = [par2[closest2[idx]] for idx in range(len(closest2))]
print('Clustering Finished')
summary



Clustering Finished


['Las proteínas se ensamblan a partir de sus aminoácidos utilizando la información codificada en los genes.',
 'Cada proteína tiene su propia secuencia de aminoácidos que está especificada por la secuencia de nucleótidos del gen que la codifica.',
 'Como el ADN contiene cuatro nucleótidos distintos, el número total de codones posibles es 64; por lo tanto, existe cierta redundancia en el código genético, estando algunos aminoácidos codificados por más de un codón.',
 'Los genes codificados en el ADN se transcriben primero en ARN pre-mensajero mediante proteínas como la ARN polimerasa.',
 'En los procariotas el ARNm puede utilizarse tan pronto como se produce, o puede unirse al ribosoma después de haberse alejado del nucleoide.',
 '[7]\u200b']

In [74]:
closest2

array([ 0,  1,  4,  5,  7, 10])